In [19]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spotify-and-youtube/Spotify_Youtube.csv


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.regularizers import l2

# load the dataset
df = pd.read_csv('/kaggle/input/spotify-and-youtube/Spotify_Youtube.csv')

# drop any song with 'Likes' value below 10,000
df = df[df['Likes'] >= 10000]

# drop empty values
df.dropna(inplace=True)

# outlier detection and removal
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1
df = df[~((df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR))).any(axis=1)]

# select the variables for regression
X = df[['Danceability', 'Energy', 'Valence', 'Acousticness', 'Loudness']]
y = df['Likes']

# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# standardize the variables
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)

# build the neural network with L2 regularization
model = Sequential()
model.add(Dense(64, input_dim=X_train_std.shape[1], activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dense(1))

# compile model
model.compile(loss='mean_squared_error', optimizer='adam')

# fit model
history = model.fit(X_train_std, y_train, epochs=100, batch_size=32, verbose=0)

# evaluate model
y_pred = model.predict(X_test_std)
print('Neural Network Regression with L2 regularization')
print('R-squared score: {:.2f}'.format(r2_score(y_test, y_pred)))
print('Mean Absolute Error: {:.2f}'.format(mean_absolute_error(y_test, y_pred)))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:25: FutureWarning: Automatic reindexing on DataFrame vs Series comparisons is deprecated and will raise ValueError in a future version.  Do `left, right = left.align(right, axis=1, copy=False)` before e.g. `left == right`


69/69 [==============================] - 0s 1ms/step
Neural Network Regression with L2 regularization
R-squared score: 0.01
Mean Absolute Error: 212950.52
